In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

import cudf
from cuml.linear_model import LinearRegression as cuLR

from glob import glob
import time
import pdb

# CuDF/CuML <=> pandas/scikit-learn

In [2]:
def run_cpu():
    start = time.time()
    df = [pd.read_csv(item) for item in glob("yellow*2020*.csv")]
    zones = pd.read_csv("taxi+_zone_lookup.csv")
    print(f"Loading: {round(time.time()-start, 6)} s")
    
    t = time.time()
    df = pd.concat(df, axis=0, ignore_index=True)
    print(f"Concatenation: {round(time.time()-t, 6)} s")
    
    t = time.time()
    top5, least5 = df.nlargest(5, 'fare_amount')['fare_amount'].tolist(), df.nsmallest(5, 'fare_amount')['fare_amount'].tolist()
    print(f"top/least 5: {top5}, {least5}")
    print(f"top/least 5: {round(time.time()-t, 6)} s")
    
    t = time.time()
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df["date"], df['hour'] = df['tpep_pickup_datetime'].dt.date, df['tpep_pickup_datetime'].dt.hour
    print(f"convert to datetime and split in date/time: {round(time.time()-t, 6)} s")
    
    t = time.time()
    grped = df.groupby(["hour", "PULocationID"]).size().to_frame('count').reset_index()
    grped = pd.merge(grped, zones, left_on='PULocationID', right_on='LocationID', how='left')
    busiest = grped[grped['hour']==8].sort_values('count')['Zone'].tail().tolist()
    print(f"Busiest PU locations: {busiest}")
    print(f"Get busiest PU location at 8 am: {round(time.time()-t, 6)} s")
    
    t = time.time()
    df = df[['passenger_count', 'trip_distance', 'congestion_surcharge', 'fare_amount']].dropna()
    reg = LinearRegression().fit(
        df[['passenger_count', 'trip_distance', 'congestion_surcharge']], df['fare_amount'])
    print(f"Linear Regression coefs: {reg.coef_}")
    print(f"Linear regressions fitted in: {round(time.time()-t, 6)} s")
    
    print(f"TOTAL RUN TIME: {time.time()-start}")
    

def run_gpu():
    start = time.time()
    df = [cudf.read_csv(item) for item in glob("yellow*2020*.csv")]
    zones = cudf.read_csv("taxi+_zone_lookup.csv")
    print(f"Loading: {round(time.time()-start, 6)} s")
    
    t = time.time()
    df = cudf.concat(df, axis=0, ignore_index=True)
    print(f"Concatenation: {round(time.time()-t, 6)} s")
    
    t = time.time()
    top5, least5 = df.nlargest(5, 'fare_amount')['fare_amount'].values.tolist(), df.nsmallest(5, 'fare_amount')['fare_amount'].values.tolist()
    print(f"top/least 5: {top5}, {least5}")
    print(f"top/least 5: {round(time.time()-t, 6)} s")
    
    t = time.time()
    df['tpep_pickup_datetime'] = cudf.to_datetime(df['tpep_pickup_datetime'])
    df["year"] = df['tpep_pickup_datetime'].dt.year
    df["month"] = df['tpep_pickup_datetime'].dt.month
    df["day"] = df['tpep_pickup_datetime'].dt.day
    df['hour'] = df['tpep_pickup_datetime'].dt.hour
    print(f"convert to datetime and split in date/time: {round(time.time()-t, 6)} s")
    
    t = time.time()
    grped = df.groupby(["hour", "PULocationID"]).size().to_frame('count').reset_index()
    grped = cudf.merge(grped, zones, left_on='PULocationID', right_on='LocationID', how='left')
    busiest = grped[grped['hour']==8].sort_values('count')['Zone'].tail().to_arrow()
    print(f"Busiest PU locations: {busiest}")
    print(f"Get busiest PU location at 8 am: {round(time.time()-t, 6)} s")
    
    t = time.time()
    df = df[['passenger_count', 'trip_distance', 'congestion_surcharge', 'fare_amount']].dropna()
    reg = cuLR().fit(
        df[['passenger_count', 'trip_distance', 'congestion_surcharge']], df['fare_amount'])
    print(f"Linear Regression coefs: {reg.coef_}")
    print(f"Linear regressions fitted in: {round(time.time()-t, 6)} s")
    
    print(f"TOTAL RUN TIME: {time.time()-start}")

In [3]:
run_cpu()

/srv/conda/envs/saturn/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading: 25.451184 s
Concatenation: 2.433759 s
top/least 5: [671100.14, 429496.72, 6052.0, 4265.0, 3014.5], [-1259.0, -1238.0, -750.0, -730.0, -500.0]
top/least 5: 8.207097 s
convert to datetime and split in date/time: 8.459456 s
Busiest PU locations: ['Murray Hill', 'Lenox Hill West', 'Penn Station/Madison Sq West', 'Upper East Side South', 'Upper East Side North']
Get busiest PU location at 8 am: 0.666884 s
Linear Regression coefs: [ 0.11428573  0.68409704 -2.43927103]
Linear regressions fitted in: 4.852894 s
TOTAL RUN TIME: 50.072824478149414


In [4]:
for i in range(2):
    run_gpu()
    print(" ")

Loading: 2.781827 s
Concatenation: 0.115865 s
top/least 5: [671100.14, 429496.72000000003, 6052.0, 4265.0, 3014.5], [-1259.0, -1238.0, -750.0, -730.0, -500.0]
top/least 5: 0.391967 s
convert to datetime and split in date/time: 0.024408 s
Busiest PU locations: [
  "Murray Hill",
  "Lenox Hill West",
  "Penn Station/Madison Sq West",
  "Upper East Side South",
  "Upper East Side North"
]
Get busiest PU location at 8 am: 0.029709 s
Linear Regression coefs: 0    0.114286
1    0.684097
2   -2.439271
dtype: float64
Linear regressions fitted in: 1.782436 s
TOTAL RUN TIME: 5.127171039581299
 
Loading: 1.04383 s
Concatenation: 0.110912 s
top/least 5: [671100.14, 429496.72000000003, 6052.0, 4265.0, 3014.5], [-1259.0, -1238.0, -750.0, -730.0, -500.0]
top/least 5: 0.387513 s
convert to datetime and split in date/time: 0.023665 s
Busiest PU locations: [
  "Murray Hill",
  "Lenox Hill West",
  "Penn Station/Madison Sq West",
  "Upper East Side South",
  "Upper East Side North"
]
Get busiest PU locat

# BlazingSQL

In [5]:
from blazingsql import BlazingContext

In [6]:
bc = BlazingContext()

bc.create_table(
    'taxidata',
    glob("yellow*2020*.csv")
)

BlazingContext ready


In [10]:
bc.sql('SELECT fare_amount FROM taxidata ORDER BY fare_amount DESC LIMIT 5')

,fare_amount
0,671100.14
1,429496.72
2,6052.00
3,4265.00
4,3014.50


In [11]:
bc.sql('SELECT fare_amount FROM taxidata ORDER BY fare_amount ASC LIMIT 5')

,fare_amount
0,-1259.0
1,-1238.0
2,-750.0
3,-730.0
4,-500.0
